In [1]:
import psycopg2
import pandas as pd

# import q_credentials.db_secmaster_cloud_cred as db_secmaster_cred
import q_credentials.db_secmaster_cred as db_secmaster_cred
import q_credentials.db_indicator_cred as db_indicator_cred

In [2]:
# connect to our securities_master database
conn_secmaster = psycopg2.connect(host=db_secmaster_cred.dbHost , database=db_secmaster_cred.dbName, user=db_secmaster_cred.dbUser, password=db_secmaster_cred.dbPWD)

In [3]:
conn_indicator = psycopg2.connect(host=db_indicator_cred.dbHost , database=db_indicator_cred.dbName, user=db_indicator_cred.dbUser, password=db_indicator_cred.dbPWD)

In [16]:
sql="""SELECT s.ticker, max(d.stock_id), max(d.date_price)
FROM public.symbol s inner join daily_data d on s.id= d.stock_id
group by s.ticker"""
df_ticker_last_day=pd.read_sql(sql,con=conn_secmaster)

In [17]:
df_ticker_last_day

,ticker,max,max
0,BOM500087,425,2020-03-27
1,BOM500010,423,2020-03-27
2,BOM500034,424,2020-03-27


In [18]:
sql="""select date_price as date, open_price as open, high_price as high, low_price as low , close_price as close,volume from daily_data where stock_id=71"""
df_stock=pd.read_sql(sql,con=conn_secmaster)

In [19]:
import datetime

In [20]:
sql="select * from indicator"
pd.read_sql(sql,con=conn_indicator)

,id,name,period,created_date,last_updated_date
0,1,candle_1,0,2020-03-31 21:05:11,2020-03-31 21:05:11
1,2,candle_2,0,2020-03-31 21:05:11,2020-03-31 21:05:11
2,3,candle_3,0,2020-03-31 21:05:11,2020-03-31 21:05:11
3,4,level,0,2020-03-31 21:05:11,2020-03-31 21:05:11
4,5,anomaly,0,2020-03-31 21:05:11,2020-03-31 21:05:11


In [26]:
symbol_id='BOM500034'
start_date=datetime.datetime(2018,1,1).strftime("%Y-%m-%d")
indicator_name = 'candle_2'

In [27]:
sql="select d.date_price as date, d.value from daily_data d join symbol s on d.symbol_id = s.id join indicator i on i.id=d.indicator_id where s.ticker='%s' and i.name = '%s' and d.date_price > '%s'" %(symbol_id, indicator_name, start_date)
df_indicator=pd.read_sql(sql,con=conn_indicator)

In [28]:
sql="select d.date_price as date, open_price as open, high_price as high, low_price as low , close_price as close,volume from daily_data d join symbol s on d.stock_id = s.id where s.ticker='%s' and d.date_price > '%s'" %(symbol_id, start_date)
df_price=pd.read_sql(sql,con=conn_secmaster)

In [29]:
df=pd.concat([df_indicator.drop(['value'], axis=1), df_indicator['value'].apply(pd.Series)], axis=1)

In [30]:
df

,date,level,pattern_name
0,2018-01-02,0.0,
1,2018-01-03,0.0,
2,2018-01-04,0.0,
3,2018-01-05,0.0,
4,2018-01-08,0.0,
...,...,...,...
321,2019-04-23,1.0,BULLISH ENGULFING
322,2019-04-24,0.0,
323,2019-04-25,-2.0,BEARISH HARAMI CROSS
324,2019-04-26,1.0,BULLISH ENGULFING
